## Import Libraries

In [1]:
import glob
from itertools import product

## Global Values

In [2]:
singularity_cmd = 'singularity exec --nv --overlay /scratch/tm3647/public/mlh-env-50G-10M.ext3:ro --overlay /scratch/tm3647/public/CheXpert_v1.sqf:ro --overlay /scratch/tm3647/public/Mimic_cxr_jpg.sqf:ro /scratch/work/public/singularity/cuda11.3.0-cudnn8-devel-ubuntu20.04.sif /bin/bash -c "'

In [3]:
cpus = "28"
mem = "128GB"

req_time = "4:30:00"

mail = "tm3647@nyu.edu"
sbatch_output_dir = "/scratch/tm3647/MLH/ConVIRT-private/sbatch_outs/"

## General Hyperparamters

In [4]:
batch_size_client_wise = {
    4: 256,
    8: 128,
    16: 64
}

In [5]:
num_rounds = [100]
num_clients = [4, 8, 16]
num_local_epochs = [1]

In [6]:
general_hyperparams = list([num_rounds, num_clients, num_local_epochs])
general_hyperparams

[[100], [4, 8, 16], [1]]

In [7]:
list(product(*general_hyperparams))

[(100, 4, 1), (100, 8, 1), (100, 16, 1)]

## Volume Configs

In [8]:
scales = [1, 10, 100, 1000]
modes = ['dirchlet']

In [9]:
volume_hyperparams = general_hyperparams+list([scales, modes])

In [10]:
volume_hyperparams

[[100], [4, 8, 16], [1], [1, 10, 100, 1000], ['dirchlet']]

In [11]:
total_exp_combinations = list(product(*volume_hyperparams))
len(total_exp_combinations), total_exp_combinations[0]

(12, (100, 4, 1, 1, 'dirchlet'))

In [12]:
env_activate_cmd = """
source /ext3/miniconda3/etc/profile.d/conda.sh;

conda activate /ext3/fedml-env/;

cd /scratch/tm3647/MLH/ConVIRT-private/;
"""

In [13]:
for nr, nc, num_l_ep, scale_val, mode_val in total_exp_combinations:
    job_name = f"fed_vol_{nr}_{nc}_{scale_val}_{mode_val}"
    sbatch_headers = f"""#!/bin/bash
    
#SBATCH --nodes=1
#SBATCH --cpus-per-task={cpus}
#SBATCH --time={req_time}
#SBATCH --mem={mem}
#SBATCH --gres=gpu:a100:1
#SBATCH --job-name={job_name}
#SBATCH --output={sbatch_output_dir}%j_0_log.out
#SBATCH --mail-type=ALL
#SBATCH --mail-user={mail}

    """
    
    py_cmd = f"\npython src/run_simulation.py --config-name=prod_simulation server_config.num_rounds={nr} pool_size={nc} partitions.num_clients={nc} partitions.scale={scale_val} partitions.mode='{mode_val}' task_name='fed_chexpert_volume' job_name={job_name} datamodule.batch_size={batch_size_client_wise[nc]}"
    
    sbatch_script = sbatch_headers + singularity_cmd + env_activate_cmd + py_cmd + "\n\""
    
    with open(f'/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/{job_name}.sbatch', 'w') as f:
        f.write(sbatch_script)

In [14]:
!cat "/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_4_1_dirchlet.sbatch"

#!/bin/bash
    
#SBATCH --nodes=1
#SBATCH --cpus-per-task=28
#SBATCH --time=4:30:00
#SBATCH --mem=128GB
#SBATCH --gres=gpu:a100:1
#SBATCH --job-name=fed_vol_100_4_1_dirchlet
#SBATCH --output=/scratch/tm3647/MLH/ConVIRT-private/sbatch_outs/%j_0_log.out
#SBATCH --mail-type=ALL
#SBATCH --mail-user=tm3647@nyu.edu

    singularity exec --nv --overlay /scratch/tm3647/public/mlh-env-50G-10M.ext3:ro --overlay /scratch/tm3647/public/CheXpert_v1.sqf:ro --overlay /scratch/tm3647/public/Mimic_cxr_jpg.sqf:ro /scratch/work/public/singularity/cuda11.3.0-cudnn8-devel-ubuntu20.04.sif /bin/bash -c "
source /ext3/miniconda3/etc/profile.d/conda.sh;

conda activate /ext3/fedml-env/;

cd /scratch/tm3647/MLH/ConVIRT-private/;

python src/run_simulation.py --config-name=prod_simulation server_config.num_rounds=100 pool_size=4 partitions.num_clients=4 partitions.scale=1 partitions.mode='dirchlet' task_name='fed_chexpert_volume' job_name=fed_vol_100_4_1_dirchlet datamodule.batch_size=256
"

In [15]:
files = glob.glob('/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/*.sbatch')

In [16]:
files

['/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_8_100_dirchlet.sbatch',
 '/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_4_100_dirchlet.sbatch',
 '/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_16_1000_dirchlet.sbatch',
 '/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_16_100_dirchlet.sbatch',
 '/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_8_10_dirchlet.sbatch',
 '/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_4_1000_dirchlet.sbatch',
 '/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_16_1_dirchlet.sbatch',
 '/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_4_10_dirchlet.sbatch',
 '/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_16_10_dirchlet.sbatch',
 '/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/vo

In [17]:
sbatch_execs = ""
for f in files:
    sbatch_execs += f"sbatch {f}\nsleep 2\n"

In [18]:
print(sbatch_execs)

sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_8_100_dirchlet.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_4_100_dirchlet.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_16_1000_dirchlet.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_16_100_dirchlet.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_8_10_dirchlet.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_4_1000_dirchlet.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_16_1_dirchlet.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_4_10_dirchlet.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vo

In [19]:
with open('/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/exec_all.sh', 'w') as f:
    f.write(sbatch_execs)

In [20]:
cat /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/exec_all.sh

sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_8_100_dirchlet.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_4_100_dirchlet.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_16_1000_dirchlet.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_16_100_dirchlet.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_8_10_dirchlet.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_4_1000_dirchlet.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_16_1_dirchlet.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vol_100_4_10_dirchlet.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/volume/fed_vo

## Class Configs

In [53]:
exclusive = [True, False]
equal_num_samples = [True, False]

In [54]:
volume_hyperparams = general_hyperparams+list([exclusive, equal_num_samples])

In [55]:
volume_hyperparams

[[100], [4, 8, 16], [1], [True, False], [True, False]]

In [56]:
total_exp_combinations = list(product(*volume_hyperparams))
len(total_exp_combinations), total_exp_combinations[0]

(12, (100, 4, 1, True, True))

In [57]:
env_activate_cmd = """
source /ext3/miniconda3/etc/profile.d/conda.sh;

conda activate /ext3/fedml-env/;

cd /scratch/tm3647/MLH/ConVIRT-private/;
"""

In [58]:
for nr, nc, num_l_ep, excl_flag, eq_num_samples_flag in total_exp_combinations:
    job_name = f"fed_class_{nr}_{nc}_{excl_flag}_{eq_num_samples_flag}"
    sbatch_headers = f"""#!/bin/bash
    
#SBATCH --nodes=1
#SBATCH --cpus-per-task={cpus}
#SBATCH --time={req_time}
#SBATCH --mem={mem}
#SBATCH --gres=gpu:a100:1
#SBATCH --job-name={job_name}
#SBATCH --output={sbatch_output_dir}%j_0_log.out
#SBATCH --mail-type=ALL
#SBATCH --mail-user={mail}

    """
    
    py_cmd = f"\npython src/run_simulation.py --config-name=prod_simulation server_config.num_rounds={nr} pool_size={nc} partitions=class partitions.num_clients={nc} partitions.exclusive={excl_flag} partitions.equal_num_samples={eq_num_samples_flag} task_name='fed_chexpert_class' job_name={job_name} datamodule.batch_size={batch_size_client_wise[nc]}"
    
    sbatch_script = sbatch_headers + singularity_cmd + env_activate_cmd + py_cmd + "\n\""
    
    with open(f'/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/class/{job_name}.sbatch', 'w') as f:
        f.write(sbatch_script)

In [59]:
!cat "/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/class/fed_class_100_16_False_False.sbatch"

#!/bin/bash
    
#SBATCH --nodes=1
#SBATCH --cpus-per-task=28
#SBATCH --time=4:30:00
#SBATCH --mem=128GB
#SBATCH --gres=gpu:a100:1
#SBATCH --job-name=fed_class_100_16_False_False
#SBATCH --output=/scratch/tm3647/MLH/ConVIRT-private/sbatch_outs/%j_0_log.out
#SBATCH --mail-type=ALL
#SBATCH --mail-user=tm3647@nyu.edu

    singularity exec --nv --overlay /scratch/tm3647/public/mlh-env-50G-10M.ext3:ro --overlay /scratch/tm3647/public/CheXpert_v1.sqf:ro --overlay /scratch/tm3647/public/Mimic_cxr_jpg.sqf:ro /scratch/work/public/singularity/cuda11.3.0-cudnn8-devel-ubuntu20.04.sif /bin/bash -c "
source /ext3/miniconda3/etc/profile.d/conda.sh;

conda activate /ext3/fedml-env/;

cd /scratch/tm3647/MLH/ConVIRT-private/;

python src/run_simulation.py --config-name=prod_simulation server_config.num_rounds=100 pool_size=16 partitions=class partitions.num_clients=16 partitions.exclusive=False partitions.equal_num_samples=False task_name='fed_chexpert_class' job_name=fed_class_100_16_False_False datamo

In [60]:
files = glob.glob('/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/class/*.sbatch')

In [61]:
files

['/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/class/fed_class_100_8_False_True.sbatch',
 '/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/class/fed_class_100_16_True_True.sbatch',
 '/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/class/fed_class_100_8_True_True.sbatch',
 '/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/class/fed_class_100_4_False_False.sbatch',
 '/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/class/fed_class_100_16_False_True.sbatch',
 '/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/class/fed_class_100_4_False_True.sbatch',
 '/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/class/fed_class_100_8_True_False.sbatch',
 '/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/class/fed_class_100_16_True_False.sbatch',
 '/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/class/fed_class_100_16_False_False.sbatch',
 '/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/class/fe

In [62]:
sbatch_execs = ""
for f in files:
    sbatch_execs += f"sbatch {f}\nsleep 2\n"

In [63]:
print(sbatch_execs)

sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/class/fed_class_100_8_False_True.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/class/fed_class_100_16_True_True.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/class/fed_class_100_8_True_True.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/class/fed_class_100_4_False_False.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/class/fed_class_100_16_False_True.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/class/fed_class_100_4_False_True.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/class/fed_class_100_8_True_False.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/class/fed_class_100_16_True_False.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/class/fed_class_100_

In [64]:
with open('/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/class/exec_all.sh', 'w') as f:
    f.write(sbatch_execs)

In [65]:
cat /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/class/exec_all.sh

sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/class/fed_class_100_8_False_True.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/class/fed_class_100_16_True_True.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/class/fed_class_100_8_True_True.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/class/fed_class_100_4_False_False.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/class/fed_class_100_16_False_True.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/class/fed_class_100_4_False_True.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/class/fed_class_100_8_True_False.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/class/fed_class_100_16_True_False.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/class/fed_class_100_

## Feature Configs

In [66]:
mode = ["nonIID", "uniform"]

In [67]:
volume_hyperparams = general_hyperparams+list([mode])

In [88]:
volume_hyperparams

[[100], [4, 8, 16], [1], ['nonIID', 'uniform']]

In [89]:
total_exp_combinations = list(product(*volume_hyperparams))
len(total_exp_combinations), total_exp_combinations[0]

(6, (100, 4, 1, 'nonIID'))

In [90]:
env_activate_cmd = """
source /ext3/miniconda3/etc/profile.d/conda.sh;

conda activate /ext3/fedml-env/;

cd /scratch/tm3647/MLH/ConVIRT-private/;
"""

In [91]:
for nr, nc, num_l_ep, mode_val in total_exp_combinations:
    job_name = f"fed_feature_{nr}_{nc}_{mode_val}"
    sbatch_headers = f"""#!/bin/bash
    
#SBATCH --nodes=1
#SBATCH --cpus-per-task={cpus}
#SBATCH --time={req_time}
#SBATCH --mem={mem}
#SBATCH --gres=gpu:a100:1
#SBATCH --job-name={job_name}
#SBATCH --output={sbatch_output_dir}%j_0_log.out
#SBATCH --mail-type=ALL
#SBATCH --mail-user={mail}

    """
    
    py_cmd = f"\npython src/run_simulation.py --config-name=prod_simulation server_config.num_rounds={nr} pool_size={nc} partitions=feature partitions.num_partitions={nc} partitions.mode={mode_val} task_name='fed_chexpert_feature' job_name={job_name} datamodule.batch_size={batch_size_client_wise[nc]}"
    
    sbatch_script = sbatch_headers + singularity_cmd + env_activate_cmd + py_cmd + "\n\""
    
    with open(f'/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/feature/{job_name}.sbatch', 'w') as f:
        f.write(sbatch_script)

In [92]:
!cat "/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/feature/fed_class_100_4_nonIID.sbatch"

#!/bin/bash
    
#SBATCH --nodes=1
#SBATCH --cpus-per-task=28
#SBATCH --time=4:30:00
#SBATCH --mem=128GB
#SBATCH --gres=gpu:a100:1
#SBATCH --job-name=fed_class_100_4_nonIID
#SBATCH --output=/scratch/tm3647/MLH/ConVIRT-private/sbatch_outs/%j_0_log.out
#SBATCH --mail-type=ALL
#SBATCH --mail-user=tm3647@nyu.edu

    singularity exec --nv --overlay /scratch/tm3647/public/mlh-env-50G-10M.ext3:ro --overlay /scratch/tm3647/public/CheXpert_v1.sqf:ro --overlay /scratch/tm3647/public/Mimic_cxr_jpg.sqf:ro /scratch/work/public/singularity/cuda11.3.0-cudnn8-devel-ubuntu20.04.sif /bin/bash -c "
source /ext3/miniconda3/etc/profile.d/conda.sh;

conda activate /ext3/fedml-env/;

cd /scratch/tm3647/MLH/ConVIRT-private/;

python src/run_simulation.py --config-name=prod_simulation server_config.num_rounds=100 pool_size=4 partitions=feature partitions.num_partitions=4 partitions.mode=nonIID task_name='fed_chexpert_feature' job_name=fed_class_100_4_nonIID datamodule.batch_size=256
"

In [96]:
files = glob.glob('/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/feature/*.sbatch')

In [97]:
files

['/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/feature/fed_feature_100_4_uniform.sbatch',
 '/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/feature/fed_feature_100_16_uniform.sbatch',
 '/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/feature/fed_feature_100_16_nonIID.sbatch',
 '/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/feature/fed_feature_100_8_uniform.sbatch',
 '/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/feature/fed_feature_100_4_nonIID.sbatch',
 '/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/feature/fed_feature_100_8_nonIID.sbatch']

In [98]:
sbatch_execs = ""
for f in files:
    sbatch_execs += f"sbatch {f}\nsleep 2\n"

In [99]:
print(sbatch_execs)

sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/feature/fed_feature_100_4_uniform.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/feature/fed_feature_100_16_uniform.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/feature/fed_feature_100_16_nonIID.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/feature/fed_feature_100_8_uniform.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/feature/fed_feature_100_4_nonIID.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/feature/fed_feature_100_8_nonIID.sbatch
sleep 2



In [100]:
with open('/scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/feature/exec_all.sh', 'w') as f:
    f.write(sbatch_execs)

In [101]:
cat /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/feature/exec_all.sh

sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/feature/fed_feature_100_4_uniform.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/feature/fed_feature_100_16_uniform.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/feature/fed_feature_100_16_nonIID.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/feature/fed_feature_100_8_uniform.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/feature/fed_feature_100_4_nonIID.sbatch
sleep 2
sbatch /scratch/tm3647/MLH/ConVIRT-private/scripts/tejas/a100/fed/feature/fed_feature_100_8_nonIID.sbatch
sleep 2


## ....